# Project: RAG with Azure SQL DB and Azure PostgreSQL

## 🔍 Objective

Demonstrate **Retrieval-Augmented Generation (RAG)** using:
- **Azure SQL Database**
- **Azure Database for PostgreSQL**
- **Azure OpenAI** embedding models

The project covers:
- Connecting to Azure OpenAI to generate embeddings using `text-embedding-3-small`
- Storing and indexing embeddings in Azure SQL and PostgreSQL
- Performing **semantic similarity search** to retrieve contextually relevant data
- Using this retrieved data for RAG-based responses

## Creating a SQL Server Database Credential

- The credential will save the Azure Open AI connection and the key to connect to the Embedding models hosted

In [ ]:

if not exists(select * from sys.symmetric_keys where [name] = '##MS_DatabaseMasterKey##')
begin
    create master key encryption by password = N'V3RYStr0NGP@ssw0rd!';
end
go
if exists(select * from sys.[database_scoped_credentials] where name = 'https://hemanthazureai.openai.azure.com')
begin
	drop database scoped credential [https://hemanthazureaiopenai.azure.com];
end
create database scoped credential [https://hemanthazureai.openai.azure.com]
with identity = 'HTTPEndpointHeaders', secret = '{"api-key": "<Key"}';
go

## Creating a stored procedure to get_embedding 

- The stored procedure will connect to the https://hemanthazureai.openai.azure.com using the above created credential and will use the embedding model;
- for this project I am going to use the text-embedding-3-small model

In [ ]:

create or alter procedure dbo.get_embedding
@deployedModelName nvarchar(1000),
@inputText nvarchar(max),
@embedding vector(1536) output
as
declare @retval int, @response nvarchar(max);
declare @payload nvarchar(max) = json_object('input': @inputText);
declare @url nvarchar(1000) = 'https://hemanthazureai.openai.azure.com/openai/deployments/' + @deployedModelName + '/embeddings?api-version=2023-03-15-preview'
exec @retval = sp_invoke_external_rest_endpoint
    @url = @url,
    @method = 'POST',
    @credential = [https://hemanthazureai.openai.azure.com],
    @payload = @payload,
    @response = @response output;

declare @re vector(1536);
if (@retval = 0) begin
    set @re = cast(json_query(@response, '$.result.data[0].embedding') as vector(1536))
end else begin
    declare @msg nvarchar(max) =  
        'Error calling OpenAI API' + char(13) + char(10) + 
        '[HTTP Status: ' + json_value(@response, '$.response.status.http.code') + '] ' +
        json_value(@response, '$.result.error.message');
    throw 50000, @msg, 1;
end

set @embedding = @re;

return @retval
go